In [ ]:
!pip install -r requirements.txt

In [16]:
import dotenv

dotenv.load_dotenv(".env")

True

# Generating the answers

## Loading questions

In [9]:
import pandas as pd
import os
df = pd.read_csv('dataset.csv', delimiter=',')
questions = df['question'].tolist()[:int(os.getenv("HOW_MANY"))]
ground_truths = df['correct'].tolist()[:int(os.getenv("HOW_MANY"))]

indeces = df['id'].tolist()[:int(os.getenv("HOW_MANY"))]

filenames = os.listdir(os.getenv("LOCAL_FILE_INPUT_DIR"))

    #filepaths = [os.path.join(os.getenv("LOCAL_FILE_INPUT_DIR"), filename) for filename in filenames if not filename.startswith(".")]
filepaths = [os.path.join(os.getenv("LOCAL_FILE_INPUT_DIR"), filename) for filename in filenames if os.path.splitext(filename)[0] in indeces]


## Indexing 

In [17]:
from haystack_integrations.components.converters.unstructured import UnstructuredFileConverter
from haystack_integrations.components.embedders.fastembed import FastembedSparseDocumentEmbedder, FastembedDocumentEmbedder
from haystack_integrations.components.retrievers.qdrant import QdrantHybridRetriever
from haystack_integrations.components.embedders.fastembed import FastembedTextEmbedder, FastembedSparseTextEmbedder
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack import Document, Pipeline

qdrant_db_sparse = QdrantDocumentStore(
    url="http://localhost:6333",  # Adjust this if your Qdrant is hosted elsewhere
    index="haystack_index",      # Use the name of your existing Qdrant index
    recreate_index=True,         # Ensure we don't overwrite the existing database
    embedding_dim=384,
    return_embedding=True,       # Return embeddings from Qdrant
    use_sparse_embeddings=True,
    sparse_idf=True
)

doc_embedder = FastembedDocumentEmbedder(model="BAAI/bge-small-en-v1.5")
doc_embedder.warm_up()
sparse_doc_embedder = FastembedSparseDocumentEmbedder(model="Qdrant/bm42-all-minilm-l6-v2-attentions")
sparse_doc_embedder.warm_up()


hybrid_indexing = Pipeline()
hybrid_indexing.add_component("converter", UnstructuredFileConverter(
    api_url="https://api.unstructuredapp.io/general/v0/general",
    document_creation_mode="one-doc-per-element"
))
hybrid_indexing.add_component("sparse_doc_embedder", FastembedSparseDocumentEmbedder(model="Qdrant/bm42-all-minilm-l6-v2-attentions"))
hybrid_indexing.add_component("dense_doc_embedder", FastembedDocumentEmbedder(model="BAAI/bge-small-en-v1.5"))
hybrid_indexing.add_component("writer", DocumentWriter(document_store=qdrant_db_sparse, policy=DuplicatePolicy.OVERWRITE))

hybrid_indexing.connect("converter", "sparse_doc_embedder")
hybrid_indexing.connect("sparse_doc_embedder", "dense_doc_embedder")
hybrid_indexing.connect("dense_doc_embedder", "writer")

hybrid_indexing.run({"paths": filepaths})

INFO: Warming up component sparse_doc_embedder...
INFO: Warming up component dense_doc_embedder...
INFO: Running component converter
Converting files to Haystack Documents: 0it [00:00, ?it/s]WARNING: 'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 402 Payment Required"
ERROR: Failed to partition the document.
ERROR: Server responded with 402 - {"detail":"Insufficient API quota for owner organization"}
{"detail":"Insufficient API quota for owner organization"}
Converting files to Haystack Documents: 1it [00:01,  1.30s/it]WARNING: 'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 402 Payment Required"
ERROR: Failed to partition set 2.
Converting files to Haystack Documents: 2it [00:03,  1.63s/

{'writer': {'documents_written': 0}}

## Retreiver

In [11]:
from haystack_integrations.components.converters.unstructured import UnstructuredFileConverter
from haystack_integrations.components.embedders.fastembed import FastembedSparseDocumentEmbedder, FastembedDocumentEmbedder
from haystack_integrations.components.retrievers.qdrant import QdrantHybridRetriever
from haystack_integrations.components.embedders.fastembed import FastembedTextEmbedder, FastembedSparseTextEmbedder
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack import Document, Pipeline
from haystack_integrations.components.generators.ollama import OllamaGenerator
import csv

qdrant_db_sparse = QdrantDocumentStore(
    url="http://localhost:6333",  # Adjust this if your Qdrant is hosted elsewhere
    index="haystack_index",      # Use the name of your existing Qdrant index
    recreate_index=False,         # Ensure we don't overwrite the existing database
    return_embedding=True,       # Return embeddings from Qdrant
    use_sparse_embeddings=True,
    sparse_idf=True,
    embedding_dim=384
)

system_prompt = "You are a helpful assistant. Answer the question based on the provided information. Answer concisely and informatively. If you don't know the answer, say so."
generation_kwargs = {
    "seed": 42,
    # "temperature": 0.8,
    # "repeat_penalty": 1.1,
    # "num_predict": 128, # max number of tokens to generate
    # "top_k": 50, # top-k sampling
    # "top_p": 0.9, # top-p sampling
    # "min_p": 0.0 # filter out token with probability less than this
}
generator = OllamaGenerator(model="llama3.2:1b",
                            url = "http://localhost:11434",
                            system_prompt=system_prompt,
                            generation_kwargs=generation_kwargs)

doc_embedder = FastembedDocumentEmbedder(model="BAAI/bge-small-en-v1.5")
doc_embedder.warm_up()
sparse_doc_embedder = FastembedSparseDocumentEmbedder(model="Qdrant/bm42-all-minilm-l6-v2-attentions")
sparse_doc_embedder.warm_up()

hybrid_query = Pipeline()
hybrid_query.add_component("sparse_text_embedder", FastembedSparseTextEmbedder(model="Qdrant/bm42-all-minilm-l6-v2-attentions"))
hybrid_query.add_component("dense_text_embedder", FastembedTextEmbedder(model="BAAI/bge-small-en-v1.5", prefix="Represent this sentence for searching relevant passages: "))
hybrid_query.add_component("retriever", QdrantHybridRetriever(document_store=qdrant_db_sparse, top_k=5))

hybrid_query.connect("sparse_text_embedder.sparse_embedding", "retriever.query_sparse_embedding")
hybrid_query.connect("dense_text_embedder.embedding", "retriever.query_embedding")

prompt_template = """
{sources_text}

{question}
"""
with open("final.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["question","ground_truth","gen_answer"])
    for i,question  in enumerate(questions):
        print(question)
        results = hybrid_query.run(
            {"dense_text_embedder": {"text": question},
            "sparse_text_embedder": {"text": question}}
        )
        sources = [result.content for result in results["retriever"]["documents"]]
        prompt = prompt_template.format(sources_text="\n\n".join(sources), question=question)

        gen_answer = generator.run(prompt, generation_kwargs=generation_kwargs)["replies"]
        writer.writerow([question, ground_truths[i], gen_answer[0]])

INFO: Warming up component sparse_text_embedder...
INFO: Warming up component dense_text_embedder...
INFO: Running component sparse_text_embedder


When was Fernando Eid born?


Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 154.33it/s]
INFO: Running component dense_text_embedder
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00, 16.62it/s]
INFO: Running component retriever
INFO: HTTP Request: GET http://localhost:6333/collections/haystack_index/exists "HTTP/1.1 200 OK"
INFO: HTTP Request: GET http://localhost:6333/collections/haystack_index "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://localhost:6333/collections/haystack_index/points/query "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO: Warming up component sparse_text_embedder...
INFO: Warming up component dense_text_embedder...
INFO: Running component sparse_text_embedder


What is the altitude range of the Zbrašov aragonite caves?


Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 177.96it/s]
INFO: Running component dense_text_embedder
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00, 19.33it/s]
INFO: Running component retriever
INFO: HTTP Request: POST http://localhost:6333/collections/haystack_index/points/query "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO: Warming up component sparse_text_embedder...
INFO: Warming up component dense_text_embedder...
INFO: Running component sparse_text_embedder


When was the new station opened?


Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 276.03it/s]
INFO: Running component dense_text_embedder
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00, 13.97it/s]
INFO: Running component retriever
INFO: HTTP Request: POST http://localhost:6333/collections/haystack_index/points/query "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
